# kaggle intent detection data pre processing

Based on code from https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings/code

word2vec file from wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

gunzip file.gz

pip install -U gensim

### things to do

1. Save and reload processed train, val and test data
2. Look at training data and indexing to embedding matrix

## imports

In [1]:
%matplotlib inline

In [2]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

In [3]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

In [4]:
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## directories and parameters

In [5]:
BASE_DIR = 'data/'
EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
TRAIN_DATA_FILE = BASE_DIR + 'train.csv'
TEST_DATA_FILE = BASE_DIR + 'test.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

In [6]:
# look into this

num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

## text to wordlist

In [7]:
# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)


### create test text lists

In [8]:
test_texts_1 = []
test_texts_2 = []
test_ids = []
with codecs.open(TEST_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        test_texts_1.append(text_to_wordlist(values[1]))
        test_texts_2.append(text_to_wordlist(values[2]))
        test_ids.append(values[0])
print('Found %s texts in test.csv' % len(test_texts_1))

Found 2345796 texts in test.csv


In [9]:
pos=10
print 'sample test data'
print '----------------'
print test_ids[pos] + '.1 - ' + test_texts_1[pos]
print test_ids[pos] + '.2 - ' + test_texts_2[pos]

sample test data
----------------
10.1 - what is the best slideshow app for android 
10.2 - what are the best app for android 


### create training text lists

In [10]:
texts_1 = [] 
texts_2 = []
labels = []
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts_1.append(text_to_wordlist(values[3]))
        texts_2.append(text_to_wordlist(values[4]))
        labels.append(int(values[5]))
print('Found %s texts in train.csv' % len(texts_1))

Found 404290 texts in train.csv


In [11]:
pos=5
print 'sample training data'
print '--------------------'
print 'text_1 - ' + texts_1[pos]
print 'text_2 - ' + texts_2[pos]
print 'intent - ' + str(labels[pos])

sample training data
--------------------
text_1 - astrology : i am a capricorn sun cap moon and cap rising what does that say about me 
text_2 - i am a triple capricorn sun moon and ascendant in capricorn what does this say about me 
intent - 1


### prepare the tokenizer word index

In [12]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_1 + texts_2 + test_texts_1 + test_texts_2)

In [13]:
word_index = tokenizer.word_index               # dictionary form - the correct form
look_word_index = list(word_index.iteritems())  # list form so I can have an easy look
print 'Found %s unique tokens' % len(word_index) 
# NOTE TO ADD CODE TO SAVE TOKENIZER FILE

Found 120500 unique tokens


In [14]:
print look_word_index[0]

('murwara', 68872)


### training dataset sequencing

In [15]:
print 'training dataset sequences'
print '--------------------------'

sequences_1 = tokenizer.texts_to_sequences(texts_1)
data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data_1 tensor:', data_1.shape)

sequences_2 = tokenizer.texts_to_sequences(texts_2)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data_2 tensor:', data_2.shape)

labels = np.array(labels)
print('Shape of label tensor:', labels.shape)

training dataset sequences
--------------------------
('Shape of data_1 tensor:', (404290, 30))
('Shape of data_2 tensor:', (404290, 30))
('Shape of label tensor:', (404290,))


### test dataset sequencing

In [16]:
print 'test dataset sequences'
print '----------------------'

test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data_1 tensor:', data_1.shape)

test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data_1 tensor:', data_1.shape)

test_ids = np.array(test_ids)
print('Shape of data_1 tensor:', data_1.shape)

test dataset sequences
----------------------
('Shape of data_1 tensor:', (404290, 30))
('Shape of data_1 tensor:', (404290, 30))
('Shape of data_1 tensor:', (404290, 30))


### make a validtion set

In [17]:
perm = np.random.permutation(len(data_1))
idx_train = perm[:int(len(data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data_1)*(1-VALIDATION_SPLIT)):]

data_1_train = np.vstack((data_1[idx_train], data_2[idx_train]))
data_2_train = np.vstack((data_2[idx_train], data_1[idx_train]))
labels_train = np.concatenate((labels[idx_train], labels[idx_train]))

data_1_val = np.vstack((data_1[idx_val], data_2[idx_val]))
data_2_val = np.vstack((data_2[idx_val], data_1[idx_val]))
labels_val = np.concatenate((labels[idx_val], labels[idx_val]))

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

In [18]:
print 'trainig data'
print '------------'
print ('Shape of data_1_train tensor:', data_1_train.shape)
print ('Shape of data_2_train tensor:', data_2_train.shape)
print ('Shape of labels_train tensor:', labels_train.shape)
print
print 'validation data'
print '---------------'
print ('Shape of data_1_val tensor:', data_1_val.shape)
print ('Shape of data_2_val tensor:', data_2_val.shape)
print ('Shape of labels_val tensor:', labels_val.shape)

trainig data
------------
('Shape of data_1_train tensor:', (727722, 30))
('Shape of data_2_train tensor:', (727722, 30))
('Shape of labels_train tensor:', (727722,))

validation data
---------------
('Shape of data_1_val tensor:', (80858, 30))
('Shape of data_2_val tensor:', (80858, 30))
('Shape of labels_val tensor:', (80858,))


### prepare embedding matrix

In [19]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

Found 3000000 word vectors of word2vec


In [20]:
nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 61789
